# 决策树（decision tree）

专家系统最常用的机器学习过程。决策树可以使用不熟悉的数据集合，并从中提取出一系列规则，机器根据数据创建规则时，就是机器学习过程。

## 算法分析

- 优点：计算复杂度不高，输出结果易于理解，对中间值缺失不敏感，可以处理不相关特征数据；
- 缺点：可能产生过度匹配的问题
- 使用数据类型：数值型和标称型

在构造决策树时，我们需要解决的第一个问题就是，当前数据集上哪个特征在划分数据分类时起决定性作用。为了找到决定性的特征，划分出最好的结果，我们
必须评估每个特征。完成测试之后，原始数据集就被划分为几个数据子集。这些数据子集会分布在第一个决策点的所有分支上。如果某个分支下的数据属于同一
类型，则当前无需阅读的垃圾邮件已经正确地划分数据分类，无需进一步对数据集进行分割。如果数据子集内的数据不属于同一类型，则需要重复划分数据子集
的过程。如何划分数据子集的算法和划分原始数据集的方法相同，直到所有具有相同类型的数据均在一个数据子集内。


## 信息增益和熵（information gain & entropy）

划分数据集的大原则是：将无序的数据变得更加有序。我们可以使用多种方法划分数据集，但是每种方法都有各自的优缺点。组织杂乱无章数据的一种方法就是
使用信息论度量信息，信息论是量化处理信息的分支科学。我们可以在划分数据前后使用信息论量化度量信息的内容。

信息的定义： $$l(x_i) = -log_2p(x_i)$$

其中$p(x_i)$是选择该分类的概率。

为了计算熵，我们需要计算所有类别所有可能值包含的信息期望值，通过下面的公式得到：

$$H = -\sum_{i=1}^n p(x_i)log_2p(x_i)$$

其中 n 是分类的数目

计算给定数据集的香农熵：

In [3]:
from math import log

def calcShannonEnt(dataSet):
    numEntries = len(dataSet)
    labelCounts = {}
    # 以下五行为所有可能分类创建字典
    for featVec in dataSet:
        currentLabel = featVec[-1]
        if currentLabel not in labelCounts.keys():
            labelCounts[currentLabel] = 0
            labelCounts[currentLabel] += 1
    shannonEnt = 0.0
    for key in labelCounts:
        prob = float(labelCounts[key]) / numEntries
        # 以 2 为底数求对数
        shannonEnt -= prob * log(prob, 2)
    return shannonEnt

def createDataSet():
    dataSet = [[1, 1, 'yes'],
              [1, 1, 'yes'],
              [1, 0, 'no'],
              [0, 1, 'no'],
              [0, 1, 'no']]
    labels = ['no surfacing', 'flippers']
    return dataSet, labels

dataset, labels = createDataSet()
calcShannonEnt(dataset)

0.9287712379549449

熵越高，则混合的数据也越多，我们下面添加更多的分类，观察熵是如何变化的。这里增加第三个名为 maybe 的分类，测试熵的变化

In [4]:
dataset[0][-1] = 'maybe'
dataset
calcShannonEnt(dataset)

1.3931568569324173

另一个度量集合无序程度的方法是基尼不纯度2 （Gini impurity），简单地说就是从一个数据集中随机选取子项，度量其被错误分类到其他分组里的概率。这里不做讨论。

得到熵之后，我们就可以按照获取最大信息增益的方法划分数据集。

### 划分数据集

分类算法除了需要测量信息熵，还需要划分数据集，度量划分数据集的熵，以便判断当前是否正确地划分了数据集。我们将对每个特征划分数据集的结果计算一次信息熵，然后判断按照哪个特征划分数据集是最好的划分方式。想象一个分布在二维空间的数据散点图，需要在数据之间划条线，将它们分成两部分，我们应该按照x轴还是y轴划线呢？答案就是本节讲述的内容。

In [5]:
def splitDataSet(dataSet, axis, value):
    # 创建新的 list 对象
    retDataSet = []
    for featVec in dataSet:
        if featVec[axis] == value:
            # 抽取
            reducedFeatVec = featVec[:axis]
            reducedFeatVec.extend(featVec[axis+1:])
            retDataSet.append(reducedFeatVec)
    return retDataSet

划分一下简单数据集 createDataSet()

In [7]:
dataset, labels = createDataSet()
splitDataSet(dataset, 0, 1)

[[1, 'yes'], [1, 'yes'], [0, 'no']]

In [8]:
splitDataSet(dataset, 0, 0)

[[1, 'no'], [1, 'no']]

接下来遍历整个数据集，循环香农熵和 splitDataSet() 函数，找到最好的特征划分方式。

In [15]:
def chooseBestFeatureToSplit(dataSet):
    numFeatures = len(dataSet[0]) - 1
    baseEntropy = calcShannonEnt(dataSet)
    bestInfoGain = 0.0;
    bestFeature = -1
    for i in range(numFeatures):
        # 创建唯一的分类标签列表
        featList = [example[i] for example in dataSet]
        uniqueVals = set(featList)
        newEntropy = 0.0
        for value in uniqueVals:
            subDataSet = splitDataSet(dataSet, i, value)
            prob = len(subDataSet) / float(len(dataSet))
            newEntropy += prob * calcShannonEnt(subDataSet)
        infoGain = baseEntropy - newEntropy
        if infoGain > bestInfoGain:
            bestInfoGain = infoGain
            bestFeature = i
    return bestFeature

测试我们的简单数据集数据划分和熵增益：

In [16]:
dataset, labels = createDataSet()
chooseBestFeatureToSplit(dataset)

1